In [1]:
import os
import json
import math
import numpy as np
import matplotlib.pyplot as plt

def find_factors_x_ge_y(z):
    z = abs(z)
    if z == 0: return 0, 0
    
    # 從平方根開始向下尋找第一個因數
    candidate = math.isqrt(z)
    
    for i in range(candidate, 0, -1):
        if z % i == 0:
            small_factor = i
            large_factor = z // i
            
            # 因為我們是從平方根往下找，large_factor 一定會 >= small_factor
            # 依照題目要求 X >= Y
            x, y = large_factor, small_factor
            return x, y
        
def plot_regret_grid(results_list, regret_key, title_prefix, filename, color, plot_type='line', is_log=False):
    """
    繪製 5x3 的 Regret 網格圖，支援對數轉換。
    """
    num_exps = len(results_list)  # 這裡設定為 15
    rows, cols = find_factors_x_ge_y(num_exps)
    fig, axes = plt.subplots(rows, cols, figsize=(18, 22))
    axes = axes.flatten()

    for i in range(num_exps):
        ax = axes[i]
        try:
            # 取得原始數據
            raw_data = np.array(results_list[i]['result'][regret_key])
            
            # --- 核心邏輯：對數轉換 ---
            if is_log:
                # 使用 np.maximum 確保數值不低於 1e-10，再取 log10
                plot_data = np.log10(np.maximum(raw_data, 1e-10))
                ylabel = "Log10(Regret)"
            else:
                plot_data = raw_data
                ylabel = "Regret Value"
            
            iters = range(1, len(plot_data) + 1)
            
            # 繪圖樣式
            if plot_type == 'line':
                ax.plot(iters, plot_data, color=color, marker='o', 
                        markersize=3, linewidth=1.2, alpha=0.8)
            else:
                ax.step(iters, plot_data, color=color, where='post', linewidth=2.5)

            # 標題優化
            file_name = os.path.basename(results_list[i].get('dataset_path', f'Exp_{i+1}'))
            ax.set_title(f"Trial {i+1}: {file_name[-20:]}", fontsize=10, fontweight='bold')
            
            # 只有在非對數模式下才畫 y=0 的線 (對數模式下 y=0 代表原始值為 1)
            if not is_log:
                ax.axhline(0, color='black', linewidth=1, linestyle='--')
            
            ax.grid(True, which='both', linestyle=':', alpha=0.6)
            
            # 座標軸標籤
            if i % cols == 0: ax.set_ylabel(ylabel)
            if i >= (rows - 1) * cols: ax.set_xlabel('Iterations')

        except Exception as e:
            ax.set_title(f"Error in Trial {i+1}")
            print(f"Error: {e}")

    # 隱藏多餘子圖
    for j in range(num_exps, len(axes)):
        axes[j].axis('off')

    # 總標題調整
    log_status = "(Log Scale)" if is_log else "(Linear Scale)"
    fig.suptitle(f"{title_prefix} {log_status}", fontsize=18, y=1.02)
    fig.suptitle(f"Comparison of {title_prefix} across {num_exps} Trials", fontsize=16, y=1.02)

    plt.tight_layout()
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    print(f"成功儲存圖表：{filename}")
    plt.close(fig)

# def plot_regret_grid(results_list, regret_key, title_prefix, filename, color, plot_type='line'):
#     """
#     一個通用的函數，用來繪製 5x3 的 Regret 網格圖並儲存。
    
#     Args:
#         results_list: 包含所有實驗結果的列表。
#         regret_key: 要畫哪種 regret ('inference_regrets' 或 'simple_regrets')。
#         title_prefix: 圖表總標題的前綴。
#         filename: 儲存的檔案名稱 (包含 .png)。
#         color: 線條顏色。
#         plot_type: 'line' (一般折線圖) 或 'step' (階梯圖)。
#     """
#     num_exps = len(results_list)  # 這裡設定為 15
#     rows, cols = find_factors_x_ge_y(num_exps)
#     num_exps = len(results_list)
#     # 設定畫布大小，figsize 可以根據需要調整
#     fig, axes = plt.subplots(rows, cols, figsize=(18, 20))
#     axes = axes.flatten()

#     for i in range(num_exps):
#         ax = axes[i]
        
#         # 安全地獲取資料
#         try:
#             regret_data = results_list[i]['result'][regret_key]
#             iters = range(1, len(regret_data) + 1)
            
#             # 根據類型選擇繪圖方式
#             if plot_type == 'line':
#                 # Inference Regret 適合用折線圖+數據點
#                 ax.plot(iters, regret_data, color=color, 
#                         marker='o', markersize=2, linestyle='-', 
#                         linewidth=1.2, alpha=0.7, label=title_prefix)
#             elif plot_type == 'step':
#                 # Simple Regret 適合用階梯圖
#                 ax.step(iters, regret_data, color=color, 
#                         where='post', linewidth=2, label=title_prefix)

#             # 獲取簡短標題
#             path_str = results_list[i].get('dataset_path', f'Trial_{i+1}')
#             short_title = os.path.basename(path_str)[-20:] # 取最後20個字元以免太長
#             ax.set_title(f"Trial {i+1}: {short_title}", fontsize=9)
            
#         except KeyError as e:
#             ax.set_title(f"Trial {i+1}: Data Error", fontsize=9, color='red')
#             print(f"Error in trial {i}: Missing key {e}")

#         # 通用樣式設定
#         ax.axhline(0, color='black', linewidth=0.8, linestyle='--')
#         ax.grid(True, which='both', linestyle=':', alpha=0.5)

#         # 座標軸標籤優化：只在最左側和最下方顯示
#         if i % cols == 0:
#             ax.set_ylabel('Regret', fontsize=10)
#         if i >= (rows - 1) * cols:
#             ax.set_xlabel('Iterations', fontsize=10)
            
#     # 隱藏多餘的空白子圖 (如果實驗少於 15 次)
#     for j in range(num_exps, len(axes)):
#         axes[j].axis('off')
        
#     # 加入一個總標題
#     fig.suptitle(f"Comparison of {title_prefix} across {num_exps} Trials", fontsize=16, y=1.02)

#     plt.tight_layout()
#     # 儲存圖片 (dpi=300 為高解析度, bbox_inches='tight' 防止邊緣被裁切)
#     print(f"Saving figure to {filename}...")
#     plt.savefig(filename, dpi=300, bbox_inches='tight')
#     plt.close(fig) # 關閉畫布釋放記憶體

In [2]:
result_path = '/workspaces/BO_EXPERIMENTS/src/results/BO_GP_EI_SLSQP_ITER30.json'
output_plot_path = './results/BO_GP_EI_SLSQP_ITER30_Log_Infer_Regret.png'
is_log = True

In [7]:
os.path.splitext(os.path.split(result_path)[-1])[1]

('BO_GP_EI_SLSQP_ITER30', '.json')

In [3]:
with open(result_path, 'r') as file:
    all_results = json.load(file)

In [4]:
# plot_regret_grid(
#     results_list=all_results,
#     regret_key='inference_regrets',
#     title_prefix='Inference Regret',
#     filename=output_plot_path,
#     color='midnightblue',  # 你要求的深一點的顏色
#     plot_type='line'
# )

# plot_regret_grid(
#     results_list=all_results,
#     regret_key='inference_regrets',
#     title_prefix='Inference Regret',
#     filename=output_plot_path,
#     color='midnightblue',  # 你要求的深一點的顏色
#     plot_type='line',
#     is_log=False
# )

plot_regret_grid(
    results_list=all_results,
    regret_key='inference_regrets',
    title_prefix='Log Inference Regret',
    filename=output_plot_path,
    color='midnightblue',  # 你要求的深一點的顏色
    plot_type='line',
    is_log=True
)

成功儲存圖表：./results/BO_GP_EI_SLSQP_ITER30_Log_Infer_Regret.png


In [5]:
# # 畫圖
# num_exps = len(results)  # 這裡設定為 15

# # 設定網格
# rows, cols = find_factors_x_ge_y(num_exps)

# # 建立畫布，figsize 寬度 18, 高度 20 (可以根據螢幕大小調整)
# fig, axes = plt.subplots(rows, cols, figsize=(18, 20))
# axes = axes.flatten()  # 將 5x3 的二維陣列拉平，方便用 for 迴圈遍歷

# for i in range(num_exps):
#     ax = axes[i]
    
#     # 這裡提取資料 (請確保 results[i] 存在)
#     inf_reg = results[i]['result']['inference_regrets']
#     sim_reg = results[i]['result']['simple_regrets']
#     iters = range(1, len(inf_reg) + 1)
    
#     # 繪製 Inference Regret (藍色小點+虛線)
#     ax.plot(
#         iters, inf_reg, 
#         label='Inference Regret', 
#         color='midnightblue',    # 改用深藍色
#         marker='o',              # 加上圓點點
#         markersize=3, 
#         linestyle='-',           # 改為實線（若要更明顯）
#         linewidth=1.2, 
#         alpha=0.5               # 提高不透明度
#     )
    
#     # 繪製 Simple Regret (紅色階梯線)
#     ax.step(
#         iters, sim_reg, 
#         label='Simple Regret (Best)', 
#         color='firebrick',       # 深紅色
#         where='post', 
#         linewidth=2.5
#     )
    
#     # 取得檔名後 15 個字元作為標題，避免太長
#     file_info = os.path.basename(results[i].get('dataset_path', f'Exp_{i+1}'))
#     ax.set_title(f"Trial {i+1}\n{file_info[-20:]}", fontsize=10)
    
#     # 圖表細節優化
#     ax.axhline(0, color='black', linewidth=0.8, linestyle='--') # 零線
#     ax.grid(True, which='both', linestyle='--', alpha=0.3)
    
#     # 只有最左邊的圖顯示 Y 軸標籤，最下方的圖顯示 X 軸標籤
#     if i % cols == 0:
#         ax.set_ylabel('Regret Value')
#     if i >= num_exps - cols:
#         ax.set_xlabel('Iterations')
        
#     # 在第一張圖顯示圖例
#     if i == 0:
#         ax.legend(loc='upper right', fontsize='small')

# # 如果實驗不足 15 次，隱藏多餘的空白子圖
# for j in range(num_exps, len(axes)):
#     axes[j].axis('off')

# # 自動調整佈局，避免標題與座標軸重疊
# plt.tight_layout()

# plt.savefig(output_plot_path, dpi=300, bbox_inches='tight')

# # 顯示圖片
# plt.show()

# # plt.close()



In [8]:
import torch

dataset = torch.load('/workspaces/BO_EXPERIMENTS/src/datasets/D=3_N=5/oracle_data_D3_A_000.pt')
dataset

{'config': {'D': 3, 'variant': 'A', 'seed': 1000},
 'initial_data': {'X': tensor([[0.2060, 0.5693, 0.2247],
          [0.1754, 0.6878, 0.1368],
          [0.1746, 0.0536, 0.7718],
          [0.2376, 0.2989, 0.4635],
          [0.7338, 0.2461, 0.0200],
          [0.1858, 0.2613, 0.5529],
          [0.5468, 0.4264, 0.0268],
          [0.6115, 0.2413, 0.1473],
          [0.0636, 0.4072, 0.5292],
          [0.8357, 0.0016, 0.1627]], dtype=torch.float64),
  'Y': tensor([[0.9360],
          [0.9672],
          [0.8758],
          [0.9165],
          [0.9225],
          [0.9193],
          [0.9550],
          [0.9251],
          [0.9175],
          [0.8818]], dtype=torch.float64)},
 'ground_truth': {'x_star': tensor([0.0000e+00, 1.0000e+00, 1.6795e-16], dtype=torch.float64),
  'f_star': 1.0115036102480213,
  'generator_seed': 1000}}